# Buzz Words but What do They Mean?

![NNs are a small subset of ai, ml, and deep learning](images\what_is_a_neural_network\small_subset_of_ai.jpg)

### Simple Neural Network

![simple neural network architecture](images\what_is_a_neural_network\basic_neural_network.jpeg)

Neural networks are really a tiny subset of a bunch of different, larger categories of problem-solving techniques.

![basic neuron in a neural network](images\what_is_a_neural_network\basic_neuron_with_bias.jpg)

The most basic unit of a neural network is a single neuron.

### What are the Parts of a Neuron?

1. inputs
2. weight & bias
3. sum (can be expressed as the dot product)
4. activation function 
5. output

### The basic Neuron class

In [236]:
class BasicNeuron:
    def __init__(self):
        self.weights = None
        self.bias = None
        
    def calc_neuron_output(self, inputs):
        sum = 0
        for inpt, weight in zip(inputs, self.weights):
            sum += inpt * weight
        sum += self.bias
        return sum 
    
        ###############
        ## IMPORTANT ##
        ###############
        # there is a much better way to do this calculation, which we will talk about. This is just for instructional purposes

Our basic Neuron class contains a few methods and a few attributes. 

It has a constructor which allows us to create Neurons.

It has a calc_neuron_output function (function and method mean the same thing) which takes in some inputs and performs the following calculation: 

\begin{equation*}
y_j = b_j +  \sum_{i} x_iw_{ij}
\end{equation*}

Which means the output of neuron "y sub j" is the sum of all the neuron's inputs times their respective weights plus a bias.

In [237]:
n = BasicNeuron()
n.weights = [0.1, 0.2, 0.3, 0.4]
n.bias = 1
inpts = [1, 1, 1, 1]

n.calc_neuron_output(inpts)

2.0

It looks like our basic Neuron class works!

### Putting a layer together

A "layer" is composed of many neurons, each with their own weights and biases.

The benefit of thinking of our neural network in terms of layers is it simplifies a lot of the calculations we must do. 

### The dot product

![types of activation functions](images\what_is_a_neural_network\dot_product_representation.png)

using the dot product instead of calculting the output for each and every neuron makes things so much easier and more efficient. 

Actually, we don't even need our BasicNeuron class if we use the dot product instead of calculating the output of each neuron one-by-one. All we need to consider is the ENTIRE layer and all of the weights and biases that belong to it.

Lets make things simpler by making a layer class.

In [238]:
import numpy as np
np.random.seed(2)

In [239]:
class Layer:
    def __init__(self, number_input_neurons = 0, number_output_neurons = 0, weights = np.array([]), biases = np.array([])):
        self.number_input_neurons = number_input_neurons
        self.number_output_neurons = number_output_neurons
        self.weights = weights
        self.biases = biases
        self.input = None
        self.output = None
        
    def initalize_random_weights(self):
        self.weights = np.random.rand(self.number_output_neurons, self.number_input_neurons)
    
    def initalize_random_biases(self):
        self.biases = np.random.rand(self.number_output_neurons, 1)
        
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.weights, self.input) + self.biases
        return self.output
    
    # We'll explain this in a bit! For now know that this is important for later.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(self.weights.T, output_error)
        weights_error = np.dot(output_error, self.input.T)

        # update parameters
        self.weights -= learning_rate * weights_error
        self.biases -= learning_rate * output_error
        return input_error

In [240]:
layer = Layer(2, 2)
layer.initalize_random_weights()
layer.initalize_random_biases()
print("weights:\n{}".format(layer.weights))
print("\n##########\n")
print("baises:\n{}".format(layer.biases))

weights:
[[0.4359949  0.02592623]
 [0.54966248 0.43532239]]

##########

baises:
[[0.4203678 ]
 [0.33033482]]


In [241]:
inputs = np.array([[0,0],
                   [0,1],
                   [1,0],
                   [1,1]])

expected_outputs = np.array([[0], 
                             [1], 
                             [1], 
                             [0]])

inputs = np.reshape(inputs, (4,-1,1))
 # rotates the entire array. We need to rotate it because we wanted the input to our NN make visual sense.
 # In most cases, you would not have to do this, but it might aid in understanding to see it this way
     #[[[0],
     # [0]],
     #
     #[[0],
     #[1]],
     #
     #[[1],
     #[0]],
     #
     #[[1],
     #[1]]]

for inpt in inputs:
    print(layer.forward_propagation(inpt))
    print("----")

[[0.4203678 ]
 [0.33033482]]
----
[[0.44629403]
 [0.76565721]]
----
[[0.8563627]
 [0.8799973]]
----
[[0.88228894]
 [1.31531969]]
----


### Types of Activation Functions

![types of activation functions](images\what_is_a_neural_network\types_of_activation_functions.jpg)

### The Sigmoid Activation Function
This is what we'll be using for this simple example:

![the sigmoid function](images\what_is_a_neural_network\sigmoid.png)

In [242]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

# we'll need the derivative of this function later!
def sigmoid_derivative(z):
    return sigmoid(z) * (1 - sigmoid(z))
    

In [243]:
# numpy allows us to apply a function to every value in the array
activation = sigmoid(layer.output)

print(layer.output)
print("\n#######\n")
print("after applying the sigmoid function to every value in the array:\n\n{}".format(activation))


[[0.88228894]
 [1.31531969]]

#######

after applying the sigmoid function to every value in the array:

[[0.70729632]
 [0.78840197]]


You will often see these activation functions being included in their own layers called the "activation layer."

Lets make an activation layer class.

In [244]:
class ActivationLayer(Layer):
    def __init__(self):
        self.output = None
        self.input = None
        
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = sigmoid(self.input)
        return self.output
    
    # Stay with us with all this backward propagation stuff. It'll make more sense in a moment.
    def backward_propagation(self, output_error, learning_rate):
        return sigmoid_derivative(self.input) * output_error

In [245]:
activation = ActivationLayer()
activation.forward_propagation(layer.output)

array([[0.70729632],
       [0.78840197]])

We can add another layer to our network with two inputs and one output


In [246]:
layer_2 = Layer(2, 1)
layer_2.initalize_random_weights()
layer_2.initalize_random_biases()
print("weights:\n{}".format(layer_2.weights))
print("\n##########\n")
print("baises:\n{}".format(layer_2.biases))

weights:
[[0.20464863 0.61927097]]

##########

baises:
[[0.29965467]]


In [247]:
layer_2.forward_propagation(activation.output)

array([[0.93263635]])

Works just about how you would expect it to!

We've created a basic Layer class which takes some inputs, has some weights and biases, and helps us "train" our neural network. We've also made an activation layer class that applies an activation function to all the data we pass into it and will also help us "train" our NN.

You might be wondering what it means to "train" a NN, and that is a very important question. 

A neural network works because we can use a little bit of calculus to adjust the weights and biases in a smart way that allows us to make our predictions more accurate. This process of adjusting the weights and biases is what it means to "train" a NN. It is ok if you are not super comfortable with the calculus, it is not super important to understand fully right now, but it might give you a better intuition into exactly what makes neural networks work. We'll keep things as simple as we can so let's dive into it.

### Measuring Error

The first step to adjusting our weights and biases is understanding how off our prediction was. After we understand how off our prediction was, we can start talking about how we can go about making our neural network perform better.

### Loss Functions (Cost Functions)

We can measure the error in our prediction using what is called a loss function (also known as a cost function). We will be using the Mean Squared Error function for this example
![MSE](images\what_is_a_neural_network\mean_squared_error.png)

In [248]:
# we take the two arrays and calculate the mse.
def mean_squared_error(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

# we'll need this later too!
def mean_squared_error_derivative(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

This gives us a measure of how off our predictions are. 

### Back Propagation

 **Explain back prop + gradient descent**

 what it is in 2d, 3d, and how to think about it in higher-dimension space

Now we have all the parts of our NN! We can put them all together in a nice little class to make things easy.

In [249]:
class NeuralNetwork:
    def __init__(self):
        self.layers = []

        
        
    def add(self, layer):
        self.layers.append(layer)
        
    def predict(self, input_data):
        
        result = []
        
        for data in input_data:
            output = data
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)
            
        return result
    
    def fit(self, x_train, y_train, no_epochs, learning_rate):
        for i in range(no_epochs):
            error = 0
            for counter, j in enumerate(x_train):
                output = j
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                
                error += mean_squared_error(y_train[counter], output)
                
                # backprop
                
                err = mean_squared_error_derivative(y_train[counter], output)
                for layer in reversed(self.layers):
                    err = layer.backward_propagation(err, learning_rate)
                    
            error /= len(x_train)
            print("epoch {}  error = {}".format(i, error))
    

In [254]:
net = NeuralNetwork()
first_layer = Layer(2,2)
first_layer.initalize_random_weights()
first_layer.initalize_random_biases()
second_layer = Layer(2,1)
second_layer.initalize_random_weights()
second_layer.initalize_random_biases()

print(first_layer.forward_propagation(np.array([[0],[0]])))

net.add(first_layer)
net.add(ActivationLayer())
net.add(second_layer)
net.add(ActivationLayer())

net.fit(inputs, expected_outputs, 1000, 1)

[[0.96455108]
 [0.50000836]]
epoch 0  error = 0.3601233947827376
epoch 1  error = 0.33316750091401304
epoch 2  error = 0.32506368671443214
epoch 3  error = 0.3229571434621671
epoch 4  error = 0.3222595286776467
epoch 5  error = 0.32183087334766347
epoch 6  error = 0.3214231662504696
epoch 7  error = 0.3209961668577034
epoch 8  error = 0.32055020325655564
epoch 9  error = 0.32009034275657733
epoch 10  error = 0.3196203615963731
epoch 11  error = 0.319142565542221
epoch 12  error = 0.31865835693398586
epoch 13  error = 0.31816866200281296
epoch 14  error = 0.31767417069122716
epoch 15  error = 0.3171754571095757
epoch 16  error = 0.31667303542990916
epoch 17  error = 0.31616738299840946
epoch 18  error = 0.3156589473766383
epoch 19  error = 0.31514814580577594
epoch 20  error = 0.31463536139669246
epoch 21  error = 0.3141209382661521
epoch 22  error = 0.3136051768106004
epoch 23  error = 0.31308832979475193
epoch 24  error = 0.31257059966635764
epoch 25  error = 0.31205213736195236
epoch

epoch 264  error = 0.2287416134820445
epoch 265  error = 0.22760671037219637
epoch 266  error = 0.22649134620330824
epoch 267  error = 0.22539656829060248
epoch 268  error = 0.2243232621500158
epoch 269  error = 0.22327215583668633
epoch 270  error = 0.22224382602481235
epoch 271  error = 0.2212387055139901
epoch 272  error = 0.22025709185088754
epoch 273  error = 0.2192991567686614
epoch 274  error = 0.21836495616770746
epoch 275  error = 0.2174544403879889
epoch 276  error = 0.21656746455329817
epoch 277  error = 0.21570379879958856
epoch 278  error = 0.2148631382314557
epoch 279  error = 0.21404511248174612
epoch 280  error = 0.2132492947782056
epoch 281  error = 0.21247521044743928
epoch 282  error = 0.21172234480983199
epoch 283  error = 0.210990150439311
epoch 284  error = 0.21027805377890624
epoch 285  error = 0.2095854611170919
epoch 286  error = 0.2089117639410883
epoch 287  error = 0.2082563436919251
epoch 288  error = 0.2076185759524194
epoch 289  error = 0.2069978341036108


epoch 556  error = 0.008915033092813529
epoch 557  error = 0.008820631012922714
epoch 558  error = 0.008728068347917556
epoch 559  error = 0.008637293941191965
epoch 560  error = 0.00854825846144857
epoch 561  error = 0.008460914323984554
epoch 562  error = 0.00837521561592467
epoch 563  error = 0.008291118025176878
epoch 564  error = 0.008208578772901197
epoch 565  error = 0.008127556549295098
epoch 566  error = 0.008048011452510806
epoch 567  error = 0.007969904930531669
epoch 568  error = 0.00789319972584488
epoch 569  error = 0.007817859822758002
epoch 570  error = 0.007743850397216246
epoch 571  error = 0.007671137768985466
epoch 572  error = 0.0075996893560743885
epoch 573  error = 0.007529473631277061
epoch 574  error = 0.007460460080723267
epoch 575  error = 0.007392619164331743
epoch 576  error = 0.007325922278066532
epoch 577  error = 0.007260341717903418
epoch 578  error = 0.007195850645417862
epoch 579  error = 0.007132423054911741
epoch 580  error = 0.00707003374200035
epo

epoch 839  error = 0.002048025099348532
epoch 840  error = 0.002042165283983617
epoch 841  error = 0.002036337607732621
epoch 842  error = 0.0020305418117648735
epoch 843  error = 0.002024777639990262
epoch 844  error = 0.0020190448390233137
epoch 845  error = 0.002013343158147882
epoch 846  error = 0.0020076723492823655
epoch 847  error = 0.002002032166945446
epoch 848  error = 0.0019964223682224086
epoch 849  error = 0.001990842712731915
epoch 850  error = 0.0019852929625933523
epoch 851  error = 0.0019797728823946177
epoch 852  error = 0.001974282239160467
epoch 853  error = 0.001968820802321301
epoch 854  error = 0.0019633883436824032
epoch 855  error = 0.0019579846373937305
epoch 856  error = 0.0019526094599200824
epoch 857  error = 0.0019472625900117317
epoch 858  error = 0.0019419438086755768
epoch 859  error = 0.0019366528991465845
epoch 860  error = 0.0019313896468598248
epoch 861  error = 0.0019261538394227818
epoch 862  error = 0.0019209452665881713
epoch 863  error = 0.0019

In [261]:
predictions = net.predict(inputs)
for inpt, expected, actual in zip(inputs, expected_outputs, predictions):
    print("input: {}, expected: {}, result: {}".format([int(inpt[0]), int(inpt[1])], expected, actual))


input: [0, 0], expected: [0], result: [[0.03202769]]
input: [0, 1], expected: [1], result: [[0.96503298]]
input: [1, 0], expected: [1], result: [[0.96415837]]
input: [1, 1], expected: [0], result: [[0.04475788]]


It works! Our Hand-coded neural network has correctly learned how to classify our inputs into the desired outputs! Maybe this doesn't seem like a huge deal, but lets scale up the project a bit.